## Experimental MPO Synchronous Endpoint

In [5]:
%run API.py
%run ENV.py
from IPython.core.display import HTML
import time
setENV(ENV)

### SyncEndpoint
Call an asynchronous MPO process and wait for it to respond.

To do this I am using a [SyncEndpoint microservice](https://github.com/MkershMambu/SyncEndpoint) hosted on AWS eu-west-2

It works but unfortunately may be too slow for most Usecases. Takes ~ 3 secs to call an MPO process to return /clients

In [88]:
%%writefile JSONBODY
{
	"methodToCall": "https://mpo-sandbox.mambucloud.com/api/1/json/public/15984/84f19977fd92ff84f48185f15f2ebc00158ae6f4",
	"params": {
		"f1": 1,
		"f2": 2
	}
	
}

Overwriting JSONBODY


In [90]:
HEADERS = {}
PARAMS = {}
s = time.perf_counter()
r = POST('http://ec2-52-56-214-201.eu-west-2.compute.amazonaws.com:8001', headers=HEADERS, params=PARAMS, body="JSONBODY")
elapsed = time.perf_counter() - s
print(f"Time Taken (secs) to Execute: {elapsed}")
PRINT(r)

Time Taken (secs) to Execute: 2.2044019809982274
API status: 200
JSON Response:
{
    "body": {
        "__conveyor_api_array__": [
            {
                "approvedDate": "2019-04-12T06:17:24+0000",
                "clientRole": {
                    "encodedKey": "8a8186a267df5bec0167df5becc90001"
                },
                "creationDate": "2019-04-12T06:17:23+0000",
                "encodedKey": "8a8187a06a101429016a102f3180001f",
                "firstName": "NewSME",
                "groupLoanCycle": 0,
                "id": "728939220",
                "lastModifiedDate": "2019-04-12T06:30:44+0000",
                "lastName": "Business",
                "loanCycle": 0,
                "notes": "<ul><li>Created this customer to test Beneficiaries Field Set</li><li>Activity feed not updated when notes added</li><li style=\"text-align: left;\">Or is it only when editing from summary</li><li style=\"text-align: left;\">Yes that appears to be the case</li><li style=\"te

### Call clients directly to see how much time we are losing

In [87]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json'}
PARAMS = {'detailsLevel': 'FULL'}
s = time.perf_counter()
r = GET('{{env1}}/clients', headers=HEADERS, params=PARAMS)
elapsed = time.perf_counter() - s
print(f"Time Taken (secs) to Execute: {elapsed}")
# PRINT(r)
printCustomers(r)

Time Taken (secs) to Execute: 0.6331887090000237
 [1] NewSME Business
 [2] Alex Yen
 [3] Currency Tester
 [4] Dominic Smith
 [5] Dominic Giles22
 [6] Harry Styles
 [7] Jim Smith
 [8] MK1 Kershaw
 [9] MPO Test Cust1 Tester3334455
 [10] Bob Diamond
 [11] Sharon Kershaw
 [12] MPO Test James
 [13] Mark Kershaw



### Run test cases (on eu-west-2)
Simulate the calling of the MPO process
Takes ~ 100ms

In [60]:
%%writefile JSONBODY
{
	"methodToCall": "http://ec2-52-56-214-201.eu-west-2.compute.amazonaws.com:8001/dummyMPOEndpoint",
	"params": {
		"f1": 1,
		"f2": 2
	}
	
}

Overwriting JSONBODY


In [65]:
HEADERS = {}
PARAMS = {}
s = time.perf_counter()
r = POST('http://ec2-52-56-214-201.eu-west-2.compute.amazonaws.com:8001', headers=HEADERS, params=PARAMS, body="JSONBODY")
elapsed = time.perf_counter() - s
print(f"Time Taken (secs) to Execute: {elapsed}")
PRINT(r)

Time Taken (secs) to Execute: 0.09923969900046359
API status: 200
JSON Response:
{
    "callbackResponseID": "b4d9a6f2-835a-4f82-8c35-61c60f6833f1",
    "msg": "Response from the MPO process called"
}


### Run test cases (on 127.0.0.1)
Simulate the calling of the MPO process
Takes ~ 8ms

In [66]:
%%writefile JSONBODY
{
	"methodToCall": "http://127.0.0.1:8001/dummyMPOEndpoint",
	"params": {
		"f1": 1,
		"f2": 2
	}
	
}

Overwriting JSONBODY


In [71]:
HEADERS = {}
PARAMS = {}
s = time.perf_counter()
r = POST('http://127.0.0.1:8001/', headers=HEADERS, params=PARAMS, body="JSONBODY")
elapsed = time.perf_counter() - s
print(f"Time Taken (secs) to Execute: {elapsed}")
PRINT(r)

Time Taken (secs) to Execute: 0.0074465289999352535
API status: 200
JSON Response:
{
    "callbackResponseID": "60328285-0f1e-4d87-a0d6-382998672404",
    "msg": "Response from the MPO process called"
}


### Run test to see how long MPO process takes to trigger

In [100]:
%%writefile JSONBODY
{
	"msg": "Send This as a Test to MPO"
	
}

Overwriting JSONBODY


In [109]:

HEADERS = {}
PARAMS = {}
s = time.perf_counter()
r = POST('https://mpo-sandbox.mambucloud.com/api/1/json/public/15985/1601d70dfa3bcde94d8a51d593940bb83047668e', headers=HEADERS, params=PARAMS, body="JSONBODY")
elapsed = time.perf_counter() - s
print(f"Time Taken (secs) to Execute: {elapsed}")
PRINT(r)

Time Taken (secs) to Execute: 1.4680328609974822
API status: 200
JSON Response:
{
    "ops": {
        "obj": "task",
        "obj_id": "5cc042c109a8af151302374f",
        "proc": "ok",
        "ref": null
    },
    "request_proc": "ok"
}
